In [24]:
import sys, os
from datetime import datetime
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_directory = '/Users/user/Projects/allee_effect/data'

def findVals(path, df=None):
    dfStart = 'Results'
    dfDate = ''
    dfTime = ''
    with open(path, 'rt', encoding = 'utf-8', errors='replace') as f:
        if df is None:
            for num, line in enumerate(f, 1):
                if dfStart in line:
                    return num
        for line in f:
            if 'Date' in line:
                dfDate = line.split('\t')[-1].strip('\n')
            if 'Time' in line:
                dfTime = line.split('\t')[-1].strip('\n')
                #dfTime = convert24(dfTime)
                    
    dT = ', '.join((dfDate, dfTime))
    return dT

def convert24(time):
    hour = str(time.split(':')[0])
    raw = str(time.split(' ')[0]).split(':', 1)[-1]
    if 'AM' in time and hour == '12':
        hour = '00'
        return hour + ':' + raw
    elif "AM" in time:
        return str(hour) + ':' + raw
    elif "PM" in time and hour == '12':
        return hour + ':' + raw
    elif "PM" in time:
        hour = str(int(hour)+12)
        return hour + ':' + raw

def addMultiIndex(df, dT):
    tempDf = df.copy()
    tempColumns = tempDf.columns
    newColumns = []
    
    for i in tempColumns:
        if i <= 6:
            newColumns.append((dT, 'MR', i))
        elif i > 6:
            newColumns.append((dT, 'MVF', i))
            
    return newColumns
    
def preprocess(data_directory):
    reads_dfs = []
    dateAndTime = []

    for dirName, subdirList, fileList in os.walk(data_directory):
        for fname in fileList:
            extension = fname.split('.')[-1]
            if extension == 'txt':
                path = os.path.join(data_directory, fname)

                df = pd.read_csv(path, skiprows=findVals(path), sep='\t', header=0, index_col=0)
                df.columns = (range(1, df.shape[1]+1))
                df = df.drop(df.columns[-1], axis=1)

                dfDateTime = findVals(path, df)
                dateAndTime.append(dfDateTime)

                #addMultiIndex(df)
                df.columns = pd.MultiIndex.from_tuples(addMultiIndex(df, dfDateTime))
                reads_dfs.append(df)

    odVals = pd.concat(reads_dfs, axis=1)
    
    return odVals, dateAndTime

odVals, dateAndTime = preprocess(data_directory)

In [47]:
def time_diff_hours (current_time, start_time):
    print(current_time, start_time)
    elapsed_time = current_time - start_time
    seconds = elapsed_time.total_seconds()
    hours = round((seconds /60 /60),2)
    return hours

def convert_to_timeDate (date, time):
    #getting the date
    first_dash = date.find("/")
    month = int(date[first_dash- 1: first_dash])
    half_date = date[first_dash + 1: len(date)]
    second_dash = half_date.find("/")
    day = int(half_date[0: second_dash])
    year = int(half_date[second_dash + 1: len(half_date)])
    #getting the time
    first_colon = time.find(":")
    initial_hour = int(time[0 : first_colon])
    half_time = time[first_colon + 1: len(time)]
    second_colon = half_time.find(":")
    minute = int(half_time[0: second_colon])
    space = half_time.find(" ")
    second = int(half_time[second_colon + 1: space])
    night = half_time[space + 1: len(half_time)]
    if night == "AM":
        if initial_hour == 12:
            hour = 0
        else:
            hour = initial_hour
    elif night == "PM":
        if initial_hour == 12:
            hour = 12
        else:
            hour = initial_hour + 12
    #get timeDate
    timeDate = datetime.datetime(year, month, day, hour, minute, second)
    return timeDate

def find_initial_time (times):
    first = min(times)
    return first

def list_of_times (DTlist):
    times = []
    i = 0
    for item in DTlist:
        comma = item.find(",")
        date = item[0: comma]
        time = item[comma + 2: len(item)]
        date_time = convert_to_timeDate(date, time)
        times.append(date_time)
    return times

#main method
def get_passedTime_array(DTlist):
    other_times = list_of_times(DTlist)
    first_time = find_initial_time(other_times)
    #other_times = list_of_times(DTlist)
    passed_time = []
    for time in other_times:
        elapsedTime = time_diff_hours(time, first_time)
        passed_time.append(elapsedTime)
    return passed_time

In [49]:
MR1to2 = odVals.loc['A',(slice(None), 'MR', 1)].values
MR1to2

array([0.358, 0.352, 0.347, 0.379, 0.35 , 0.256])

In [48]:
get_passedTime_array(dateAndTime)

2019-08-08 16:50:59 2019-08-06 12:48:16
2019-08-09 09:52:50 2019-08-06 12:48:16
2019-08-08 09:40:03 2019-08-06 12:48:16
2019-08-07 17:04:39 2019-08-06 12:48:16
2019-08-07 10:17:30 2019-08-06 12:48:16
2019-08-06 12:48:16 2019-08-06 12:48:16


[52.05, 69.08, 44.86, 28.27, 21.49, 0.0]